# Evolutionary Game Theory on Multilayer Networks

This notebook provides functions for computing structure coefficients (θ and φ) that characterize the conditions for cooperation to be favored in evolutionary games on multilayer networks.

## Overview

We implement four update rule combinations:
- **dB-dB**: Death-Birth update on both layers
- **dB-Bd**: Death-Birth on layer 1, Birth-Death on layer 2

Each can be computed under two fitness schemes:
- **PF (payoff-fecundity)**: Standard scheme
- **FF (fecundity-fecundity)**: Alternative fitness computation

## Input Format

### Edge Sequence
A matrix where each row is `[source_node, target_node, edge_weight, layer_id]`:
- Nodes must be labeled 1, 2, ..., N
- Layers must be labeled 1, 2, ..., L
- For undirected edges, include both directions: `[i, j, w, ℓ]` and `[j, i, w, ℓ]`
- Edge weights must be positive integers

### Strategy Configuration
A matrix where each row is `[node_id, strategy, layer_id]`:
- Strategy 1 = Cooperator (mutant)
- Strategy 0 = Defector (resident)

## Output Format

The functions return a 4×L matrix `theta_phi` where:
- `theta_phi[:, 1] = [0, θ₁, θ₂, θ₃]` — strategy assortment within layer 1
- `theta_phi[:, ℓ] = [0, φ₀₁, φ₂₀, φ₂₁]` — strategy assortment between layer 1 and layer ℓ

## References

https://github.com/qisu1991/MultilayerPopulations<br>
See the accompanying manuscript for theoretical details.

In [3]:
using LinearAlgebra
using SparseArrays
using IterativeSolvers

---
## Utility Functions

In [6]:
"""
    edge_seq_rearrangement(edge_seq::Array{Int64,2}) -> Array{Int64,2}

Rearrange edge sequence in increasing order of layer ID and starting node ID.
This preprocessing step ensures consistent ordering for downstream computations.

# Arguments
- `edge_seq`: Edge sequence matrix [source, target, weight, layer]

# Returns
- Rearranged edge sequence matrix
"""
function edge_seq_rearrangement(edge_seq::Array{Int64,2})
    layer_list = setdiff(edge_seq[:, 4], [])
    L = maximum(layer_list)
    if length(setdiff([i for i = 1:L], layer_list)) != 0
        println("Warning: Please label layers in the order 1, 2, ..., L")
    end
    
    node_list = setdiff(edge_seq[:, 1], [])
    N = maximum(edge_seq[:, 1])
    if length(setdiff([i for i = 1:N], node_list)) != 0
        println("Warning: Please label nodes in the order 1, 2, ..., N")
    end

    edge_seq_rearranged = zeros(Int64, size(edge_seq))
    id = 1
    for ell in layer_list
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], edge_seq[edge_list_ell, 3])
        inds = findall(!iszero, M)
        a = getindex.(inds, 1)
        b = getindex.(inds, 2)
        edge_seq_rearranged[id:id+length(a)-1, 1] = b
        edge_seq_rearranged[id:id+length(a)-1, 2] = a
        edge_seq_rearranged[id:id+length(a)-1, 3] = M[inds]
        edge_seq_rearranged[id:id+length(a)-1, 4] = ell * ones(Int64, length(a))
        id = id + length(a)
    end

    return edge_seq_rearranged
end

edge_seq_rearrangement

---
## Network Generation Utilities

Helper functions for generating standard network structures.

In [9]:
"""
    generate_complete_graph(N::Int, L::Int) -> Array{Int64,2}

Generate a complete graph on N nodes replicated across L layers.

# Arguments
- `N`: Number of nodes
- `L`: Number of layers

# Returns
- Edge sequence matrix for complete multilayer network
"""
function generate_complete_graph(N::Int, L::Int)
    edge_seq = Matrix{Int64}(undef, 0, 4)
    for layer in 1:L
        for i in 1:N
            for j in 1:N
                if i != j
                    edge_seq = vcat(edge_seq, reshape([i, j, 1, layer], 1, 4))
                end
            end
        end
    end
    return edge_seq
end

"""
    generate_ring_network(N::Int, L::Int) -> Array{Int64,2}

Generate a ring (cycle) network on N nodes replicated across L layers.

# Arguments
- `N`: Number of nodes
- `L`: Number of layers

# Returns
- Edge sequence matrix for ring multilayer network
"""
function generate_ring_network(N::Int, L::Int)
    edge_seq = Matrix{Int64}(undef, 0, 4)
    for layer in 1:L
        for i in 1:N
            next_node = mod(i, N) + 1
            edge_seq = vcat(edge_seq, reshape([i, next_node, 1, layer], 1, 4))
            edge_seq = vcat(edge_seq, reshape([next_node, i, 1, layer], 1, 4))
        end
    end
    return edge_seq
end

"""
    generate_strategy_configuration(N::Int, L::Int; mutant_node::Int=1) -> Array{Int64,2}

Generate a strategy configuration with a single mutant (cooperator) at the specified node.

# Arguments
- `N`: Number of nodes
- `L`: Number of layers
- `mutant_node`: Node ID for the mutant (default: 1)

# Returns
- Strategy configuration matrix [node_id, strategy, layer_id]
"""
function generate_strategy_configuration(N::Int, L::Int; mutant_node::Int=1)
    xi_seq = Matrix{Int64}(undef, 0, 3)
    for layer in 1:L
        for i in 1:N
            strategy = (i == mutant_node) ? 1 : 0
            xi_seq = vcat(xi_seq, reshape([i, strategy, layer], 1, 3))
        end
    end
    return xi_seq
end

generate_strategy_configuration

---
## Death-Birth / Death-Birth (dB-dB) Update Rule

Both layers use the Death-Birth update:
1. A random node is selected to die (proportional to degree)
2. Neighbors compete to fill the vacancy (proportional to fitness)

In [12]:
"""
    beta_gamma_dB_dB(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2}) -> SparseMatrixCSC

Compute the β and γ coefficients for the dB-dB update rule.

Solves the system of equations to obtain:
- β_ij: Pairwise strategy assortment within layer 1
- γ_ij: Cross-layer strategy assortment between layer 1 and other layers

# Arguments
- `edge_seq`: Preprocessed edge sequence matrix
- `xi_seq`: Strategy configuration matrix

# Returns
- Solution matrix where column 1 contains β_ij and column ℓ contains γ_ij for layer ℓ
"""
function beta_gamma_dB_dB(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    N = maximum(edge_seq[:, 1])
    L = maximum(edge_seq[:, 4])
    
    # Build adjacency matrices for all layers
    M = spzeros(Float64, N * L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M_ell = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], 
                       edge_seq[edge_list_ell, 3], N, N)
        M[(ell-1)*N+1:ell*N, :] = M_ell
    end
    
    # Compute stationary distribution and normalize transition matrices
    pi = spzeros(Float64, L * N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:, 1]
        pi_transient = spzeros(Float64, N)
        presence = findall(!iszero, pi_2)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient

        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] = M_transient[presence, :] ./ pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    # Extract strategy vector
    xi = spzeros(Float64, L * N)
    for ell = 1:L
        xi_transient = spzeros(Int64, N)
        presence = findall(isequal(ell), xi_seq[:, 3])
        xi_transient[xi_seq[presence, 1]] = xi_seq[presence, 2]
        xi[(ell-1)*N+1:ell*N] = xi_transient
    end

    # Solve for β_i
    presence_list = findall(!iszero, pi[1:N])
    N1 = length(presence_list)
    absence_list = findall(iszero, pi[1:N])
    MatA_i = M[1:N, :] - sparse(Matrix(1.0I, N, N))
    pi_list = findall(!isequal(presence_list[1]), [i for i = 1:N])
    MatA_i[:, pi_list] = MatA_i[:, pi_list] - MatA_i[:, presence_list[1]] * 
                         transpose(pi[pi_list] / pi[presence_list[1]])
    MatA_i_reduced = MatA_i[pi_list, pi_list]

    xi_hat = sum(pi[1:N] .* xi[1:N])
    MatB_i = (xi_hat * ones(Float64, N) - xi[1:N]) * N1
    MatB_i_reduced = MatB_i[pi_list]

    beta_i_solution_reduced = idrs(MatA_i_reduced, MatB_i_reduced)
    beta_i_solution = zeros(Float64, N)
    beta_i_solution[pi_list] = beta_i_solution_reduced
    beta_i_solution[presence_list[1]] = -sum(pi[1:N] .* beta_i_solution) / pi[presence_list[1]]

    # Solve for β_ij
    Mat1 = M[1:N, :]
    inds = findall(!iszero, Mat1)
    a = getindex.(inds, 1)
    b = getindex.(inds, 2)
    vec1 = transpose([i for i = 1:N])
    X1 = N * (a .- 1) * ones(Int64, 1, N) + ones(Int64, length(a)) * vec1
    Y1 = N * (b .- 1) * ones(Int64, 1, N) + ones(Int64, length(b)) * vec1
    W1 = Mat1[inds] * ones(Int64, 1, N)

    vec2 = transpose([(i-1)*N for i = 1:N])
    X2 = a * ones(Int64, 1, N) + ones(Int64, length(a)) * vec2
    Y2 = b * ones(Int64, 1, N) + ones(Int64, length(b)) * vec2
    W2 = Mat1[inds] * ones(Int64, 1, N)

    X11 = reshape(X1, :, 1)
    Y11 = reshape(Y1, :, 1)
    W11 = reshape(W1, :, 1)
    X22 = reshape(X2, :, 1)
    Y22 = reshape(Y2, :, 1)
    W22 = reshape(W2, :, 1)
    
    MatA_ij = sparse(X11[:, 1], Y11[:, 1], W11[:, 1], N^2, N^2) / 2 + 
              sparse(X22[:, 1], Y22[:, 1], W22[:, 1], N^2, N^2) / 2
    MatA_ij = MatA_ij - sparse(Matrix(1.0I, N * N, N * N))
    
    beta_obtained = [(i-1)*N+i for i = 1:N]
    beta_missed = setdiff([i for i = 1:N^2], beta_obtained)
    MatA_ij_reduced = MatA_ij[beta_missed, beta_missed]

    MatB_ij = -sum(MatA_ij[:, beta_obtained] .* transpose(beta_i_solution), dims=2)
    Mat = xi_hat * ones(Float64, N, N) * N1 / 2 - xi[1:N] * transpose(xi[1:N]) * N1 / 2
    Mat[absence_list, :] = spzeros(Float64, length(absence_list), N)
    Mat[:, absence_list] = spzeros(Float64, N, length(absence_list))
    MatB_ij = MatB_ij + reshape(transpose(Mat), :)
    MatB_ij_reduced = MatB_ij[beta_missed]

    beta_ij_solution_reduced = idrs(MatA_ij_reduced, MatB_ij_reduced)
    beta_ij_solution = zeros(Float64, N * N)
    beta_ij_solution[beta_missed] = beta_ij_solution_reduced
    beta_ij_solution[beta_obtained] = beta_i_solution

    solution = spzeros(Float64, N * N, L)
    solution[:, 1] = beta_ij_solution
    
    # Solve for γ_ij (cross-layer assortment)
    if L > 1
        for ell = 2:L
            xi_ell = xi[(ell-1)*N+1:ell*N]
            pi_ell = pi[(ell-1)*N+1:ell*N]
            presence_ell = findall(!iszero, pi_ell)
            N_ell = length(presence_ell)
            
            GammaA_ij = sparse(X11[:, 1], Y11[:, 1], W11[:, 1], N^2, N^2) * 
                        (N_ell - 1) / (N1 + N_ell - 1)

            Mat_ell = M[(ell-1)*N+1:ell*N, :]
            inds_ell = findall(!iszero, Mat_ell)
            a_ell = getindex.(inds_ell, 1)
            b_ell = getindex.(inds_ell, 2)
            vec_ell = transpose([(i-1)*N for i = 1:N])
            X_ell = a_ell * ones(Int64, 1, N) + ones(Int64, length(a_ell)) * vec_ell
            Y_ell = b_ell * ones(Int64, 1, N) + ones(Int64, length(b_ell)) * vec_ell
            W_ell = Mat_ell[inds_ell] * ones(Int64, 1, N)
            X_ellell = reshape(X_ell, :, 1)
            Y_ellell = reshape(Y_ell, :, 1)
            W_ellell = reshape(W_ell, :, 1)
            
            GammaA_ij = GammaA_ij + sparse(X_ellell[:, 1], Y_ellell[:, 1], W_ellell[:, 1], N^2, N^2) * 
                        (N1 - 1) / (N1 + N_ell - 1)

            vec_gamma1 = reshape(ones(Int64, N) * vec1, :)
            vec_gamma2 = reshape(transpose(vec1) * ones(Int64, 1, N), :)
            X_gamma1 = N * (a .- 1) * ones(Int64, 1, N*N) + ones(Int64, length(a)) * transpose(vec_gamma1)
            Y_gamma1 = N * (b .- 1) * ones(Int64, 1, N*N) + ones(Int64, length(b)) * transpose(vec_gamma2)
            W_gamma1 = Mat1[inds] * ones(Int64, 1, N*N)

            vec_gamma3 = [(i-1)*N for i in vec_gamma1]
            vec_gamma4 = [(i-1)*N for i in vec_gamma2]
            X_gamma2 = a_ell * ones(Int64, 1, N*N) + ones(Int64, length(a_ell)) * transpose(vec_gamma3)
            Y_gamma2 = b_ell * ones(Int64, 1, N*N) + ones(Int64, length(b_ell)) * transpose(vec_gamma4)
            W_gamma2 = Mat_ell[inds_ell] * ones(Int64, 1, N*N)

            X_gamma_11 = reshape(X_gamma1, :, 1)
            Y_gamma_11 = reshape(Y_gamma1, :, 1)
            W_gamma_11 = reshape(W_gamma1, :, 1)
            X_gamma_22 = reshape(X_gamma2, :, 1)
            Y_gamma_22 = reshape(Y_gamma2, :, 1)
            W_gamma_22 = reshape(W_gamma2, :, 1)
            
            GammaA_ij = GammaA_ij + sparse(X_gamma_11[:, 1], Y_gamma_11[:, 1], W_gamma_11[:, 1], N^2, N^2) .* 
                        sparse(X_gamma_22[:, 1], Y_gamma_22[:, 1], W_gamma_22[:, 1], N^2, N^2) / (N1 + N_ell - 1)

            GammaA_ij = GammaA_ij - sparse(Matrix(1.0I, N * N, N * N))
            pi_transient = pi[1:N] .* pi_ell
            presence_list_gamma = findall(!iszero, pi_transient)
            absence_list_gamma = findall(iszero, pi_transient)
            GammaA_delete = (presence_list_gamma .- 1) * N + presence_list_gamma
            GammaA_ij[:, GammaA_delete] = GammaA_ij[:, GammaA_delete] - 
                                          GammaA_ij[:, GammaA_delete[1]] * 
                                          transpose(pi[presence_list_gamma]) / pi[presence_list_gamma[1]]
            pi_list_gamma = findall(!isequal(GammaA_delete[1]), [i for i = 1:N*N])
            GammaA_ij_reduced = GammaA_ij[pi_list_gamma, pi_list_gamma]

            xi_ell_hat = sum(pi_ell .* xi_ell)
            GammaB_ij = (xi_hat * xi_ell_hat * ones(Float64, N, N) - xi[1:N] * transpose(xi_ell)) * 
                        N1 * N_ell / (N1 + N_ell - 1)
            presence1 = findall(iszero, pi[1:N])[:, 1]
            presence_ell_zero = findall(iszero, pi_ell)[:, 1]
            GammaB_ij[presence1, :] = spzeros(Float64, length(presence1), N)
            GammaB_ij[:, presence_ell_zero] = spzeros(Float64, N, length(presence_ell_zero))
            GammaB_ij = reshape(transpose(GammaB_ij), :)
            GammaB_ij_reduced = GammaB_ij[pi_list_gamma]

            gamma_ij_solution_reduced = idrs(GammaA_ij_reduced, GammaB_ij_reduced)
            gamma_ij_solution = zeros(Float64, N * N)
            gamma_ij_solution[pi_list_gamma] = gamma_ij_solution_reduced
            gamma_ij_solution[GammaA_delete[1]] = -sum(pi[presence_list_gamma] .* 
                                                       gamma_ij_solution[GammaA_delete]) / pi[presence_list_gamma[1]]
            solution[:, ell] = gamma_ij_solution
        end
    end

    return solution
end

beta_gamma_dB_dB

In [14]:
"""
    bc_multilayer_dB_dB_pf(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2}) -> Matrix{Float64}

Compute structure coefficients θ and φ for dB-dB update rule under the PF (payoff-fecundity) scheme.

# Arguments
- `edge_seq`: Edge sequence matrix [source, target, weight, layer]
- `xi_seq`: Strategy configuration matrix [node_id, strategy, layer_id]

# Returns
- 4×L matrix where:
  - Column 1: [0, θ₁, θ₂, θ₃] (within-layer assortment)
  - Column ℓ: [0, φ₀₁, φ₂₀, φ₂₁] (cross-layer assortment with layer ℓ)
"""
function bc_multilayer_dB_dB_pf(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma_dB_dB(edge_seq, xi_seq)
    
    N = maximum(edge_seq[:, 1])
    L = maximum(edge_seq[:, 4])
    
    # Rebuild adjacency and stationary distribution
    M = spzeros(Float64, N * L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M_ell = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], 
                       edge_seq[edge_list_ell, 3], N, N)
        M[(ell-1)*N+1:ell*N, :] = M_ell
    end
    
    pi = spzeros(Float64, L * N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:, 1]
        pi_transient = spzeros(Float64, N)
        presence = findall(!iszero, pi_2)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient

        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] = M_transient[presence, :] ./ pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    # Compute θ coefficients
    M1 = M[1:N, :]
    beta = transpose(reshape(solution[:, 1], N, :))
    theta1 = sum(pi[1:N] .* M1 .* beta)
    M2 = M1 * M1
    theta2 = sum(pi[1:N] .* M2 .* beta)
    M3 = M2 * M1
    theta3 = sum(pi[1:N] .* M3 .* beta)

    theta_phi = zeros(Float64, 4, L)
    theta_phi[:, 1] = [0.0, theta1, theta2, theta3]

    # Compute φ coefficients for each additional layer
    if L > 1
        for ell = 2:L
            gamma = transpose(reshape(solution[:, ell], N, :))
            M_ell = M[(ell-1)*N+1:ell*N, :]
            phi01 = sum(pi[1:N] .* M_ell .* gamma)
            phi20 = sum(pi[1:N] .* M2 .* gamma)
            M_ell21 = M2 * M_ell
            phi21 = sum(pi[1:N] .* M_ell21 .* gamma)
            theta_phi[:, ell] = [0, phi01, phi20, phi21]
        end
    end

    return theta_phi
end

bc_multilayer_dB_dB_pf

In [16]:
"""
    bc_multilayer_dB_dB_ff(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2}) -> Matrix{Float64}

Compute structure coefficients θ and φ for dB-dB update rule under the FF (fecundity-fecundity) scheme.

The FF scheme differs from PF in how θ₁ and θ₃ are computed, using the transpose of M1.

# Arguments
- `edge_seq`: Edge sequence matrix [source, target, weight, layer]
- `xi_seq`: Strategy configuration matrix [node_id, strategy, layer_id]

# Returns
- 4×L matrix of structure coefficients
"""
function bc_multilayer_dB_dB_ff(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma_dB_dB(edge_seq, xi_seq)
    
    N = maximum(edge_seq[:, 1])
    L = maximum(edge_seq[:, 4])
    
    M = spzeros(Float64, N * L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M_ell = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], 
                       edge_seq[edge_list_ell, 3], N, N)
        M[(ell-1)*N+1:ell*N, :] = M_ell
    end
    
    pi = spzeros(Float64, L * N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:, 1]
        pi_transient = spzeros(Float64, N)
        presence = findall(!iszero, pi_2)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient

        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] = M_transient[presence, :] ./ pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    # FF scheme: use transpose of M1 for θ₁ and θ₃
    M1 = M[1:N, :]
    beta = transpose(reshape(solution[:, 1], N, :))
    theta1 = sum(pi[1:N] .* transpose(M1) .* beta)  # FF modification
    M2 = M1 * M1
    theta2 = sum(pi[1:N] .* M2 .* beta)
    M3_prime = M2 * transpose(M1)  # FF modification
    theta3 = sum(pi[1:N] .* M3_prime .* beta)

    theta_phi = zeros(Float64, 4, L)
    theta_phi[:, 1] = [0.0, theta1, theta2, theta3]

    if L > 1
        for ell = 2:L
            gamma = transpose(reshape(solution[:, ell], N, :))
            M_ell = M[(ell-1)*N+1:ell*N, :]
            phi01 = sum(pi[1:N] .* M_ell .* gamma)
            phi20 = sum(pi[1:N] .* M2 .* gamma)
            M_ell21 = M2 * M_ell
            phi21 = sum(pi[1:N] .* M_ell21 .* gamma)
            theta_phi[:, ell] = [0, phi01, phi20, phi21]
        end
    end

    return theta_phi
end

bc_multilayer_dB_dB_ff

---
## Death-Birth / Birth-Death (dB-Bd) Update Rule

Layer 1 uses Death-Birth, Layer 2 uses Birth-Death:
- **Death-Birth**: Random death → neighbors compete to reproduce
- **Birth-Death**: Random birth (proportional to fitness) → random neighbor dies

In [19]:
"""
    beta_gamma_dB_Bd(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2}) -> SparseMatrixCSC

Compute the β and γ coefficients for the dB-Bd update rule.

The key difference from dB-dB is in how the cross-layer transition matrices are constructed,
accounting for the Birth-Death dynamics in layer 2.

# Arguments
- `edge_seq`: Preprocessed edge sequence matrix
- `xi_seq`: Strategy configuration matrix

# Returns
- Solution matrix with β_ij in column 1 and γ_ij for each layer in subsequent columns
"""
function beta_gamma_dB_Bd(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    N = maximum(edge_seq[:, 1])
    L = maximum(edge_seq[:, 4])
    
    M = spzeros(Float64, N * L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M_ell = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], 
                       edge_seq[edge_list_ell, 3], N, N)
        M[(ell-1)*N+1:ell*N, :] = M_ell
    end
    
    pi = spzeros(Float64, L * N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:, 1]
        pi_transient = spzeros(Float64, N)
        presence = findall(!iszero, pi_2)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient

        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] = M_transient[presence, :] ./ pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    xi = spzeros(Float64, L * N)
    for ell = 1:L
        xi_transient = spzeros(Int64, N)
        presence = findall(isequal(ell), xi_seq[:, 3])
        xi_transient[xi_seq[presence, 1]] = xi_seq[presence, 2]
        xi[(ell-1)*N+1:ell*N] = xi_transient
    end

    # Solve for β_i (same as dB-dB)
    presence_list = findall(!iszero, pi[1:N])
    N1 = length(presence_list)
    absence_list = findall(iszero, pi[1:N])
    MatA_i = M[1:N, :] - sparse(Matrix(1.0I, N, N))
    pi_list = findall(!isequal(presence_list[1]), [i for i = 1:N])
    MatA_i[:, pi_list] = MatA_i[:, pi_list] - MatA_i[:, presence_list[1]] * 
                         transpose(pi[pi_list] / pi[presence_list[1]])
    MatA_i_reduced = MatA_i[pi_list, pi_list]

    xi_hat = sum(pi[1:N] .* xi[1:N])
    MatB_i = (xi_hat * ones(Float64, N) - xi[1:N]) * N1
    MatB_i_reduced = MatB_i[pi_list]

    beta_i_solution_reduced = idrs(MatA_i_reduced, MatB_i_reduced)
    beta_i_solution = zeros(Float64, N)
    beta_i_solution[pi_list] = beta_i_solution_reduced
    beta_i_solution[presence_list[1]] = -sum(pi[1:N] .* beta_i_solution) / pi[presence_list[1]]

    # Solve for β_ij (same as dB-dB)
    Mat1 = M[1:N, :]
    inds = findall(!iszero, Mat1)
    a = getindex.(inds, 1)
    b = getindex.(inds, 2)
    vec1 = transpose([i for i = 1:N])
    X1 = N * (a .- 1) * ones(Int64, 1, N) + ones(Int64, length(a)) * vec1
    Y1 = N * (b .- 1) * ones(Int64, 1, N) + ones(Int64, length(b)) * vec1
    W1 = Mat1[inds] * ones(Int64, 1, N)

    vec2 = transpose([(i-1)*N for i = 1:N])
    X2 = a * ones(Int64, 1, N) + ones(Int64, length(a)) * vec2
    Y2 = b * ones(Int64, 1, N) + ones(Int64, length(b)) * vec2
    W2 = Mat1[inds] * ones(Int64, 1, N)

    X11 = reshape(X1, :, 1)
    Y11 = reshape(Y1, :, 1)
    W11 = reshape(W1, :, 1)
    X22 = reshape(X2, :, 1)
    Y22 = reshape(Y2, :, 1)
    W22 = reshape(W2, :, 1)
    
    MatA_ij = sparse(X11[:, 1], Y11[:, 1], W11[:, 1], N^2, N^2) / 2 + 
              sparse(X22[:, 1], Y22[:, 1], W22[:, 1], N^2, N^2) / 2
    MatA_ij = MatA_ij - sparse(Matrix(1.0I, N * N, N * N))
    
    beta_obtained = [(i-1)*N+i for i = 1:N]
    beta_missed = setdiff([i for i = 1:N^2], beta_obtained)
    MatA_ij_reduced = MatA_ij[beta_missed, beta_missed]

    MatB_ij = -sum(MatA_ij[:, beta_obtained] .* transpose(beta_i_solution), dims=2)
    Mat = xi_hat * ones(Float64, N, N) * N1 / 2 - xi[1:N] * transpose(xi[1:N]) * N1 / 2
    Mat[absence_list, :] = spzeros(Float64, length(absence_list), N)
    Mat[:, absence_list] = spzeros(Float64, N, length(absence_list))
    MatB_ij = MatB_ij + reshape(transpose(Mat), :)
    MatB_ij_reduced = MatB_ij[beta_missed]

    beta_ij_solution_reduced = idrs(MatA_ij_reduced, MatB_ij_reduced)
    beta_ij_solution = zeros(Float64, N * N)
    beta_ij_solution[beta_missed] = beta_ij_solution_reduced
    beta_ij_solution[beta_obtained] = beta_i_solution

    solution = spzeros(Float64, N * N, L)
    solution[:, 1] = beta_ij_solution
    
    # Solve for γ_ij with Birth-Death dynamics in layer 2
    if L > 1
        for ell = 2:L
            xi_ell = xi[(ell-1)*N+1:ell*N]
            pi_ell = pi[(ell-1)*N+1:ell*N]
            presence_ell = findall(!iszero, pi_ell)
            N_ell = length(presence_ell)
            current_N = N

            # Birth-Death specific: column sums for diagonal scaling
            sum_col_M = vec(sum(M[(ell-1)*N+1:ell*N, :], dims=1))

            i_indices = repeat(1:current_N, inner=current_N)
            j_indices = repeat(1:current_N, outer=current_N)
            D_new_values = current_N .- sum_col_M[j_indices]
            linear_indices = (j_indices .- 1) .* current_N .+ i_indices
            D_new = sparse(linear_indices, linear_indices, D_new_values, current_N^2, current_N^2)

            GammaA_ij = D_new * sparse(X11[:, 1], Y11[:, 1], W11[:, 1], current_N^2, current_N^2)

            Mat_ell = transpose(M[(ell-1)*N+1:ell*N, :])
            inds_ell = findall(!iszero, Mat_ell)
            a_ell = getindex.(inds_ell, 1)
            b_ell = getindex.(inds_ell, 2)
            vec_ell = transpose([(i-1)*N for i = 1:current_N])
            X_ell = a_ell * ones(Int64, 1, current_N) + ones(Int64, length(a_ell)) * vec_ell
            Y_ell = b_ell * ones(Int64, 1, current_N) + ones(Int64, length(b_ell)) * vec_ell
            W_ell = Mat_ell[inds_ell] * ones(Int64, 1, current_N)
            X_ellell = reshape(X_ell, :, 1)
            Y_ellell = reshape(Y_ell, :, 1)
            W_ellell = reshape(W_ell, :, 1)

            GammaA_ij += sparse(X_ellell[:, 1], Y_ellell[:, 1], W_ellell[:, 1], current_N^2, current_N^2) * (N1 - 1)

            # Second diagonal scaling
            D_new_values2 = current_N .+ (N - 1) .* sum_col_M[j_indices]
            D_new2 = sparse(linear_indices, linear_indices, D_new_values2, current_N^2, current_N^2)
            GammaA_ij = GammaA_ij - D_new2

            pi_transient = pi[1:current_N] .* pi_ell
            presence_list_gamma = findall(!iszero, pi_transient)
            absence_list_gamma = findall(iszero, pi_transient)

            GammaA_delete = (presence_list_gamma .- 1) * current_N .+ presence_list_gamma
            GammaA_ij[:, GammaA_delete] .= GammaA_ij[:, GammaA_delete] .- 
                                           GammaA_ij[:, GammaA_delete[1]] * 
                                           transpose(pi[presence_list_gamma]) / pi[presence_list_gamma[1]]

            pi_list_gamma = findall(i -> i != GammaA_delete[1], 1:current_N^2)
            GammaA_ij_reduced = GammaA_ij[pi_list_gamma, pi_list_gamma]

            xi_ell_hat = sum(pi_ell .* xi_ell)
            GammaB_ij = (xi_hat * xi_ell_hat * ones(Float64, current_N, current_N) - 
                         xi[1:current_N] * transpose(xi_ell)) * N1 * N_ell
            presence1 = findall(iszero, pi[1:current_N])[:, 1]
            presence_ell_zero = findall(iszero, pi_ell)[:, 1]
            GammaB_ij[presence1, :] .= spzeros(Float64, length(presence1), current_N)
            GammaB_ij[:, presence_ell_zero] .= spzeros(Float64, current_N, length(presence_ell_zero))
            GammaB_ij = reshape(transpose(GammaB_ij), :)
            GammaB_ij_reduced = GammaB_ij[pi_list_gamma]

            gamma_ij_solution_reduced = idrs(GammaA_ij_reduced, GammaB_ij_reduced)
            gamma_ij_solution = zeros(Float64, N * N)
            gamma_ij_solution[pi_list_gamma] = gamma_ij_solution_reduced
            gamma_ij_solution[GammaA_delete[1]] = -sum(pi[presence_list_gamma] .* 
                                                       gamma_ij_solution[GammaA_delete]) / pi[presence_list_gamma[1]]
            solution[:, ell] = gamma_ij_solution
        end
    end

    return solution
end

beta_gamma_dB_Bd

In [21]:
"""
    bc_multilayer_dB_Bd_pf(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2}) -> Matrix{Float64}

Compute structure coefficients θ and φ for dB-Bd update rule under the PF scheme.

# Arguments
- `edge_seq`: Edge sequence matrix [source, target, weight, layer]
- `xi_seq`: Strategy configuration matrix [node_id, strategy, layer_id]

# Returns
- 4×L matrix of structure coefficients
"""
function bc_multilayer_dB_Bd_pf(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma_dB_Bd(edge_seq, xi_seq)
    
    N = maximum(edge_seq[:, 1])
    L = maximum(edge_seq[:, 4])
    
    M = spzeros(Float64, N * L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M_ell = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], 
                       edge_seq[edge_list_ell, 3], N, N)
        M[(ell-1)*N+1:ell*N, :] = M_ell
    end
    
    pi = spzeros(Float64, L * N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:, 1]
        pi_transient = spzeros(Float64, N)
        presence = findall(!iszero, pi_2)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient

        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] = M_transient[presence, :] ./ pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    M1 = M[1:N, :]
    beta = transpose(reshape(solution[:, 1], N, :))
    theta1 = sum(pi[1:N] .* M1 .* beta)
    M2 = M1 * M1
    theta2 = sum(pi[1:N] .* M2 .* beta)
    M3 = M2 * M1
    theta3 = sum(pi[1:N] .* M3 .* beta)

    theta_phi = zeros(Float64, 4, L)
    theta_phi[:, 1] = [0.0, theta1, theta2, theta3]

    if L > 1
        for ell = 2:L
            gamma = transpose(reshape(solution[:, ell], N, :))
            M_ell = M[(ell-1)*N+1:ell*N, :]
            phi01 = sum(pi[1:N] .* M_ell .* gamma)
            phi20 = sum(pi[1:N] .* M2 .* gamma)
            M_ell21 = M2 * M_ell
            phi21 = sum(pi[1:N] .* M_ell21 .* gamma)
            theta_phi[:, ell] = [0, phi01, phi20, phi21]
        end
    end

    return theta_phi
end

bc_multilayer_dB_Bd_pf

In [23]:
"""
    bc_multilayer_dB_Bd_ff(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2}) -> Matrix{Float64}

Compute structure coefficients θ and φ for dB-Bd update rule under the FF scheme.

# Arguments
- `edge_seq`: Edge sequence matrix [source, target, weight, layer]
- `xi_seq`: Strategy configuration matrix [node_id, strategy, layer_id]

# Returns
- 4×L matrix of structure coefficients
"""
function bc_multilayer_dB_Bd_ff(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma_dB_Bd(edge_seq, xi_seq)
    
    N = maximum(edge_seq[:, 1])
    L = maximum(edge_seq[:, 4])
    
    M = spzeros(Float64, N * L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:, 4])
        M_ell = sparse(edge_seq[edge_list_ell, 1], edge_seq[edge_list_ell, 2], 
                       edge_seq[edge_list_ell, 3], N, N)
        M[(ell-1)*N+1:ell*N, :] = M_ell
    end
    
    pi = spzeros(Float64, L * N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:, 1]
        pi_transient = spzeros(Float64, N)
        presence = findall(!iszero, pi_2)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient

        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] = M_transient[presence, :] ./ pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    # FF scheme modifications
    M1 = M[1:N, :]
    beta = transpose(reshape(solution[:, 1], N, :))
    theta1 = sum(pi[1:N] .* transpose(M1) .* beta)  # FF modification
    M2 = M1 * M1
    theta2 = sum(pi[1:N] .* M2 .* beta)
    M3_prime = M2 * transpose(M1)  # FF modification
    theta3 = sum(pi[1:N] .* M3_prime .* beta)

    theta_phi = zeros(Float64, 4, L)
    theta_phi[:, 1] = [0.0, theta1, theta2, theta3]

    if L > 1
        for ell = 2:L
            gamma = transpose(reshape(solution[:, ell], N, :))
            M_ell = M[(ell-1)*N+1:ell*N, :]
            phi01 = sum(pi[1:N] .* M_ell .* gamma)
            phi20 = sum(pi[1:N] .* M2 .* gamma)
            M_ell21 = M2 * M_ell
            phi21 = sum(pi[1:N] .* M_ell21 .* gamma)
            theta_phi[:, ell] = [0, phi01, phi20, phi21]
        end
    end

    return theta_phi
end

bc_multilayer_dB_Bd_ff

---
## Example Usage

Demonstration with a simple two-layer network.

In [26]:
# Example: Star network in layer 1, Complete graph in layer 2
# 6 nodes, 2 layers

edge_seq = [1 6 1 1;   # Layer 1: Star with center at node 6
            2 6 1 1;
            3 6 1 1;
            4 6 1 1;
            5 6 1 1;
            6 1 1 1;
            6 2 1 1;
            6 3 1 1;
            6 4 1 1;
            6 5 1 1;
            1 2 1 2;   # Layer 2: Star with center at node 1
            1 3 1 2;
            1 4 1 2;
            1 5 1 2;
            1 6 1 2;
            2 1 1 2;
            3 1 1 2;
            4 1 1 2;
            5 1 1 2;
            6 1 1 2]

# Strategy: Mutant (cooperator) at node 1 in layer 1, node 6 in layer 2
xi_seq = [1 1 1;   # Node 1 is cooperator in layer 1
          2 0 1;
          3 0 1;
          4 0 1;
          5 0 1;
          6 0 1;
          1 0 2;
          2 0 2;
          3 0 2;
          4 0 2;
          5 0 2;
          6 1 2]   # Node 6 is cooperator in layer 2

println("Computing structure coefficients for example network...")
println()

# dB-dB with PF scheme
theta_phi_dBdB_pf = bc_multilayer_dB_dB_pf(edge_seq, xi_seq)
println("dB-dB (PF scheme):")
println("  θ = ", theta_phi_dBdB_pf[:, 1])
println("  φ = ", theta_phi_dBdB_pf[:, 2])
println()

# dB-dB with FF scheme
theta_phi_dBdB_ff = bc_multilayer_dB_dB_ff(edge_seq, xi_seq)
println("dB-dB (FF scheme):")
println("  θ = ", theta_phi_dBdB_ff[:, 1])
println("  φ = ", theta_phi_dBdB_ff[:, 2])
println()

# dB-Bd with PF scheme
theta_phi_dBBd_pf = bc_multilayer_dB_Bd_pf(edge_seq, xi_seq)
println("dB-Bd (PF scheme):")
println("  θ = ", theta_phi_dBBd_pf[:, 1])
println("  φ = ", theta_phi_dBBd_pf[:, 2])
println()

# dB-Bd with FF scheme  
theta_phi_dBBd_ff = bc_multilayer_dB_Bd_ff(edge_seq, xi_seq)
println("dB-Bd (FF scheme):")
println("  θ = ", theta_phi_dBBd_ff[:, 1])
println("  φ = ", theta_phi_dBBd_ff[:, 2])

Computing structure coefficients for example network...

dB-dB (PF scheme):
[0.0, -0.8999999999999996, -0.5999999999999999, -0.8999999999999996]
  φ = [0.0, -0.10516363636365691, -0.016363636363641576, -0.1411636363636512]

dB-dB (FF scheme):
  θ = [0.0, -2.3400000000000265, -0.600000000000003, -2.3400000000000265]
  φ = [0.0, -0.10516363636365161, -0.016363636363644726, -0.14116363636364268]

dB-Bd (PF scheme):
  θ = [0.0, -0.8999999999999433, -0.6000000000000235, -0.8999999999999433]
  φ = [0.0, 0.4240888385342263, 0.23142360912238016, 0.17958917946860725]

dB-Bd (FF scheme):
  θ = [0.0, -2.339999999999772, -0.5999999999999797, -2.339999999999772]
  φ = [0.0, 0.4240888385344006, 0.23142360912229817, 0.1795891794688572]


In [27]:
# Example: Ring network with 10 nodes, 2 layers

N = 10
L = 2

edge_seq_ring = generate_ring_network(N, L)
xi_seq_ring = generate_strategy_configuration(N, L; mutant_node=1)

println("Ring network with N=$N nodes, L=$L layers")
println()

theta_phi = bc_multilayer_dB_dB_pf(edge_seq_ring, xi_seq_ring)
println("dB-dB (PF scheme) for ring:")
println("  θ = ", theta_phi[:, 1])
println("  φ = ", theta_phi[:, 2])

Ring network with N=10 nodes, L=2 layers

dB-dB (PF scheme) for ring:
  θ = [0.0, -4.499999999999998, -4.0, -5.999999999999999]
  φ = [0.0, -0.4770462766452819, -0.4131670194602474, -0.6268729062292195]


In [28]:
# Example: Complete graph with 5 nodes, 2 layers

N = 5
L = 2

edge_seq_complete = generate_complete_graph(N, L)
xi_seq_complete = generate_strategy_configuration(N, L; mutant_node=1)

println("Complete graph with N=$N nodes, L=$L layers")
println()

theta_phi = bc_multilayer_dB_dB_pf(edge_seq_complete, xi_seq_complete)
println("dB-dB (PF scheme) for complete graph:")
println("  θ = ", theta_phi[:, 1])
println("  φ = ", theta_phi[:, 2])

Complete graph with N=5 nodes, L=2 layers

dB-dB (PF scheme) for complete graph:
  θ = [0.0, -2.000000000000001, -1.5000000000000009, -1.6250000000000007]
  φ = [0.0, -0.4571428571428573, -0.34285714285714297, -0.3714285714285715]


---
## API Summary

### Main Functions

| Function | Update Rule | Fitness Scheme |
|----------|-------------|----------------|
| `bc_multilayer_dB_dB_pf` | dB-dB | PF  |
| `bc_multilayer_dB_dB_ff` | dB-dB | FF  |
| `bc_multilayer_dB_Bd_pf` | dB-Bd | PF  |
| `bc_multilayer_dB_Bd_ff` | dB-Bd | FF  |

### Helper Functions

- `edge_seq_rearrangement(edge_seq)` — Preprocess edge sequence
- `generate_complete_graph(N, L)` — Create complete multilayer network
- `generate_ring_network(N, L)` — Create ring multilayer network
- `generate_strategy_configuration(N, L; mutant_node=1)` — Create strategy config

### Internal Functions

- `beta_gamma_dB_dB(edge_seq, xi_seq)` — Solve for β, γ coefficients (dB-dB)
- `beta_gamma_dB_Bd(edge_seq, xi_seq)` — Solve for β, γ coefficients (dB-Bd)